In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

## Загрузка данных

In [2]:
data = pd.read_csv('./data/amine_treatment-final.csv', on_bad_lines='skip')
data

,"feed_gas temperature, C","feed_gas pressure, kPa","feed_gas mass flow, kg/h",feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,...,degased_amine Propane mol frac,degased_amine i-Butane mol frac,degased_amine n-Butane mol frac,degased_amine i-Pentane mol frac,degased_amine n-Pentane mol frac,degased_amine H2S mol frac,degased_amine H2O mol frac,degased_amine MDEAmine mol frac,degased_amine H2S ppm,degased_amine CO2 ppm
0,52.0,6701.325,259647.0,24.668523,80.705482,0.000000,3217.216383,0.031875,0.633122,0.192729,...,0.000004,1.979803e-06,6.785930e-06,3.039502e-08,1.629357e-07,0.014838,0.910916,0.062270,22521.880092,23245.607009
1,40.0,6701.325,63853.0,23.411939,79.292250,0.000000,805.286774,0.039520,0.681947,0.179992,...,0.000006,1.099164e-07,7.886635e-07,2.425456e-06,2.679624e-08,0.010470,0.896612,0.077190,15891.219056,30468.484150
2,46.0,6701.325,15928.0,27.803853,108.358884,0.000000,146.993024,0.027859,0.577870,0.163095,...,0.000018,5.179434e-05,2.863001e-05,4.721545e-06,1.359401e-06,0.001284,0.860268,0.132676,1949.372531,10648.306282
3,36.0,6701.325,150276.0,22.053344,74.092459,0.000000,2028.222609,0.005420,0.856717,0.000249,...,0.000039,8.734358e-06,3.706189e-05,5.426945e-05,3.949909e-06,0.001233,0.809722,0.184359,1871.528366,8438.736973
4,12.0,6701.325,52691.0,24.334510,114.182302,0.002870,461.460940,0.027297,0.694634,0.091063,...,0.000100,7.719691e-05,2.661312e-05,1.351350e-05,6.619552e-06,0.003300,0.788340,0.200715,5008.207346,14011.150105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512795,18.0,6701.325,278813.0,27.267819,147.437736,0.239690,1890.816202,0.036621,0.666807,0.042159,...,0.000053,1.226987e-04,3.336251e-05,4.145371e-06,8.510407e-06,0.002101,0.796328,0.187149,3188.443718,27411.897443
512796,33.0,6701.325,72612.0,26.629816,112.530303,0.000000,645.266193,0.025383,0.605142,0.150361,...,0.000049,8.271254e-05,1.199104e-05,1.810930e-06,4.749955e-06,0.003784,0.826966,0.162727,5742.847844,11941.343733
512797,17.0,6701.325,272718.0,26.290335,135.390018,0.133398,2014.180541,0.039356,0.606850,0.167318,...,0.000037,2.874297e-05,3.343060e-06,1.045919e-05,2.187381e-06,0.045335,0.725301,0.179546,68812.636471,97071.672175
512798,59.0,6701.325,75970.0,24.231673,73.727565,0.000000,1030.415146,0.037951,0.732049,0.086286,...,0.000013,1.782460e-05,8.286743e-06,5.417825e-06,2.393693e-07,0.002551,0.877567,0.106280,3871.441495,26287.366187


# Работаем над определением физических свойств потоков

In [210]:
stream_physic = data.sample(frac=0.85)
len(stream_physic)

435880

In [211]:
list(data.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

#### Получаем обучающую и тестовую выборку для определения физических свойств сырьевого газа

In [212]:
feed_gas_physic = stream_physic[['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                                 'feed_gas molecular weight', 'feed_gas Mass density, kg/m3', 'feed_gas actual liquid flow, m3/h',
                                 'feed_gas actual vapour flow, m3/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
                                 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                                 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                                 'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac'
                                ]]

# Создаем X и y
X = feed_gas_physic[['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                    'feed_gas CO2 mol frac', 'feed_gas Methane mol frac','feed_gas Ethane mol frac',
                    'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
                    ]]

y = feed_gas_physic.drop(['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                    'feed_gas CO2 mol frac', 'feed_gas Methane mol frac','feed_gas Ethane mol frac',
                    'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
                    ], axis=1)

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                    'feed_gas CO2 mol frac', 'feed_gas Methane mol frac','feed_gas Ethane mol frac',
                    'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac'
                     ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)
print(X_test_normal.shape)
print(y_test.shape)

(87176, 14)
(87176, 4)


#### Нейронная сеть для определения состава продуктов

In [213]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
feed_gas_physic_model = tf.keras.Sequential([
    tf.keras.layers.Dense(14, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(4)
])

# 2. Compile the model
feed_gas_physic_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['mae']
)

# 3. Fit the model
feed_gas_physic_model.fit(X_train_normal, y_train, epochs=50)

Epoch 1/50
10897/10897 [==============================] - 36s 3ms/step - loss: 58.2497 - mae: 58.2497
Epoch 2/50
10897/10897 [==============================] - 31s 3ms/step - loss: 9.4778 - mae: 9.4778
Epoch 3/50
10897/10897 [==============================] - 36s 3ms/step - loss: 5.7488 - mae: 5.7488
Epoch 4/50
10897/10897 [==============================] - 33s 3ms/step - loss: 4.7452 - mae: 4.7452
Epoch 5/50
10897/10897 [==============================] - 34s 3ms/step - loss: 3.8138 - mae: 3.8138
Epoch 6/50
10897/10897 [==============================] - 34s 3ms/step - loss: 2.8004 - mae: 2.8004
Epoch 7/50
10897/10897 [==============================] - 35s 3ms/step - loss: 2.2180 - mae: 2.2180
Epoch 8/50
10897/10897 [==============================] - 38s 3ms/step - loss: 1.9040 - mae: 1.9040
Epoch 9/50
10897/10897 [==============================] - 37s 3ms/step - loss: 1.6975 - mae: 1.6975
Epoch 10/50
10897/10897 [==============================] - 37s 3ms/step - loss: 1.5631 - mae: 1.56

In [214]:
feed_gas_physic_model.evaluate(X_test_normal)

2725/2725 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


[0.0, 0.0]

In [215]:
y_pred = feed_gas_physic_model.predict(X_test_normal)

2725/2725 [==============================] - 6s 2ms/step


In [221]:
print(y_pred[3])

[ 2.4020823e+01  9.2877922e+01 -1.2657841e-02  9.5154413e+02]


In [217]:
y_test

,feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h"
136171,26.319260,122.601660,0.093627,1723.047802
227551,25.626912,83.672483,0.000000,1142.394684
88168,26.346754,89.034395,0.000000,804.576702
90095,23.968643,92.912640,0.000000,950.753307
189167,23.622452,91.875312,0.013504,672.920266
...,...,...,...,...
312758,27.445647,98.235808,0.000000,981.118817
235816,26.542611,116.677651,0.000000,2079.635626
67406,24.576183,77.904430,0.000000,710.216350
242807,28.177109,170.295262,0.054123,477.216935


# Работаем над определением составов продуктовых потоков

### Получаем отдельную выборку данных для определения составов продуктов

In [50]:
rounded_composition = data
columns = [ 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane mol frac',
 'lean_amine n-Butane mol frac',
 'lean_amine i-Pentane mol frac',
 'lean_amine n-Pentane mol frac',
 'lean_amine H2S mol frac',
 'lean_amine H2O mol frac',
 'lean_amine MDEAmine mol frac',
 'sweet_gas CO2 mol frac',
 'sweet_gas Methane mol frac',
 'sweet_gas Ethane mol frac',
 'sweet_gas Propane mol frac',
 'sweet_gas i-Butane mol frac',
 'sweet_gas n-Butane mol frac',
 'sweet_gas i-Pentane mol frac',
 'sweet_gas n-Pentane mol frac',
 'sweet_gas H2S mol frac',
 'sweet_gas H2O mol frac',
 'sweet_gas MDEAmine mol frac',
 'rich_amine CO2 mol frac',
 'rich_amine Methane mol frac',
 'rich_amine Ethane mol frac',
 'rich_amine Propane mol frac',
 'rich_amine i-Butane mol frac',
 'rich_amine n-Butane mol frac',
 'rich_amine i-Pentane mol frac',
 'rich_amine n-Pentane mol frac',
 'rich_amine H2S mol frac',
 'rich_amine H2O mol frac',
 'rich_amine MDEAmine mol frac']

for col in columns:
    rounded_composition[col] = rounded_composition[col].apply(lambda x: 0 if x < 0.0001 else x)

In [52]:
rounded_composition_frac = rounded_composition.sample(frac=0.8)
len(rounded_composition_frac)

410240

#### Получаем обучающую и тестовую выборку

In [76]:
mol_composition = rounded_composition_frac[['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane mol frac',
 'lean_amine n-Butane mol frac',
 'lean_amine i-Pentane mol frac',
 'lean_amine n-Pentane mol frac',
 'lean_amine H2S mol frac',
 'lean_amine H2O mol frac',
 'lean_amine MDEAmine mol frac',
 'lean_amine H2S ppm',
 'lean_amine CO2 ppm',
'sweet_gas temperature, C',
'sweet_gas mass flow, kg/h',
'sweet_gas CO2 mol frac',
 'sweet_gas Methane mol frac',
 'sweet_gas Ethane mol frac',
 'sweet_gas Propane mol frac',
 'sweet_gas i-Butane mol frac',
 'sweet_gas n-Butane mol frac',
 'sweet_gas i-Pentane mol frac',
 'sweet_gas n-Pentane mol frac',
 'sweet_gas H2S mol frac',
 'sweet_gas H2O mol frac',
 'sweet_gas MDEAmine mol frac',
 'sweet_gas H2S ppm',
    'sweet_gas CO2 ppm',
 'rich_amine temperature, C',
 'rich_amine mass flow, kg/h', 
 'rich_amine CO2 mol frac',
 'rich_amine Methane mol frac',
 'rich_amine Ethane mol frac',
 'rich_amine Propane mol frac',
 'rich_amine i-Butane mol frac',
 'rich_amine n-Butane mol frac',
 'rich_amine i-Pentane mol frac',
 'rich_amine n-Pentane mol frac',
 'rich_amine H2S mol frac',
 'rich_amine H2O mol frac',
 'rich_amine MDEAmine mol frac',
 'rich_amine H2S ppm',
 'rich_amine CO2 ppm',]]

# Создаем X и y
stream_names_to_remove = ['sweet_gas', 'rich_amine']
pattern = '|'.join(stream_names_to_remove)
cols_to_remove = mol_composition.filter(regex=pattern).columns

X = mol_composition.drop(cols_to_remove, axis=1)
y = mol_composition[['sweet_gas CO2 mol frac',
 'sweet_gas Methane mol frac',
 'sweet_gas Ethane mol frac',
 'sweet_gas Propane mol frac',
 'sweet_gas i-Butane mol frac',
 'sweet_gas n-Butane mol frac',
 'sweet_gas i-Pentane mol frac',
 'sweet_gas n-Pentane mol frac',
 'sweet_gas H2S mol frac',
 'sweet_gas H2O mol frac',
 'sweet_gas MDEAmine mol frac',
 'rich_amine CO2 mol frac',
 'rich_amine Methane mol frac',
 'rich_amine Ethane mol frac',
 'rich_amine Propane mol frac',
 'rich_amine i-Butane mol frac',
 'rich_amine n-Butane mol frac',
 'rich_amine i-Pentane mol frac',
 'rich_amine n-Pentane mol frac',
 'rich_amine H2S mol frac',
 'rich_amine H2O mol frac',
 'rich_amine MDEAmine mol frac']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane mol frac',
 'lean_amine n-Butane mol frac',
 'lean_amine i-Pentane mol frac',
 'lean_amine n-Pentane mol frac',
 'lean_amine H2S mol frac',
 'lean_amine H2O mol frac',
 'lean_amine MDEAmine mol frac',
 'lean_amine H2S ppm',
 'lean_amine CO2 ppm',])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)
X_train_normal

array([[0.24      , 0.        , 0.0394214 , ..., 0.44271448, 0.02951684,
        0.67205729],
       [0.84      , 0.        , 0.47175585, ..., 0.12709431, 0.02943025,
        0.22303811],
       [0.78      , 0.        , 0.50061538, ..., 0.60666474, 0.69155892,
        0.16843319],
       ...,
       [0.32      , 0.        , 0.07473244, ..., 0.2111647 , 0.24861689,
        0.44678276],
       [0.32      , 0.        , 0.82580268, ..., 0.10368038, 0.53163949,
        0.34287566],
       [0.42      , 0.        , 0.28399666, ..., 0.80083676, 0.88323555,
        0.67422117]])

#### Нейронная сеть для определения состава продуктов

In [77]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_composition_model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(22)
])

# 2. Compile the model
prod_composition_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    metrics=['mae']
)

# 3. Fit the model
prod_composition_model.fit(X_train_normal, y_train, epochs=10)

Epoch 1/10
10256/10256 [==============================] - 25s 2ms/step - loss: 0.0183 - mae: 0.0183
Epoch 2/10
10256/10256 [==============================] - 22s 2ms/step - loss: 0.0032 - mae: 0.0032
Epoch 3/10
10256/10256 [==============================] - 30s 3ms/step - loss: 0.0021 - mae: 0.0021
Epoch 4/10
10256/10256 [==============================] - 28s 3ms/step - loss: 0.0018 - mae: 0.0018
Epoch 5/10
10256/10256 [==============================] - 24s 2ms/step - loss: 0.0017 - mae: 0.0017
Epoch 6/10
10256/10256 [==============================] - 25s 2ms/step - loss: 0.0016 - mae: 0.0016
Epoch 7/10
10256/10256 [==============================] - 26s 3ms/step - loss: 0.0015 - mae: 0.0015
Epoch 8/10
10256/10256 [==============================] - 27s 3ms/step - loss: 0.0014 - mae: 0.0014
Epoch 9/10
10256/10256 [==============================] - 27s 3ms/step - loss: 0.0014 - mae: 0.0014
Epoch 10/10
10256/10256 [==============================] - 28s 3ms/step - loss: 0.0013 - mae: 0.0013

In [78]:
prod_composition_model.evaluate(X_test_normal)

2564/2564 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


[0.0, 0.0]

In [81]:
y_pred = prod_composition_model.predict(X_test_normal)
print(y_pred[10])
print(' ')
print(sum(y_pred[10]))

2564/2564 [==============================] - 4s 2ms/step
[ 1.2630204e-02  6.4978427e-01  1.7491040e-01  8.4930010e-02
  1.0974254e-03  7.0709184e-02  3.3002011e-03  2.7896296e-03
 -1.8793368e-04  1.0562213e-03 -1.2951728e-04  1.6741136e-02
  1.3516501e-03 -1.9483967e-05  6.9271307e-05 -1.8142629e-05
  1.9887462e-05 -2.3483811e-04 -3.7720194e-05  1.3597662e-02
  7.8428727e-01  1.8103164e-01]
 
1.9976784273749217


In [75]:
y_test

,sweet_gas CO2 mol frac,sweet_gas Methane mol frac,sweet_gas Ethane mol frac,sweet_gas Propane mol frac,sweet_gas i-Butane mol frac,sweet_gas n-Butane mol frac,sweet_gas i-Pentane mol frac,sweet_gas n-Pentane mol frac,sweet_gas H2S mol frac,sweet_gas H2O mol frac,...,rich_amine Methane mol frac,rich_amine Ethane mol frac,rich_amine Propane mol frac,rich_amine i-Butane mol frac,rich_amine n-Butane mol frac,rich_amine i-Pentane mol frac,rich_amine n-Pentane mol frac,rich_amine H2S mol frac,rich_amine H2O mol frac,rich_amine MDEAmine mol frac
142117,0.004916,0.616092,0.188781,0.011283,0.096964,0.047708,0.023745,0.008814,0.0,0.001626,...,0.001783,0.001367,0.0,0.001142,0.0,0.0,0.0,0.003404,0.693726,0.292650
204727,0.002279,0.858628,0.064145,0.019068,0.001579,0.025850,0.024487,0.001643,0.0,0.002305,...,0.001813,0.000000,0.0,0.000000,0.0,0.0,0.0,0.007573,0.808524,0.142131
72330,0.012279,0.752498,0.100052,0.034351,0.017800,0.065314,0.011516,0.004503,0.0,0.001675,...,0.001303,0.000000,0.0,0.000000,0.0,0.0,0.0,0.011834,0.874660,0.100194
482160,0.003028,0.723551,0.041124,0.102147,0.073212,0.033652,0.015350,0.007025,0.0,0.000000,...,0.001766,0.000000,0.0,0.000000,0.0,0.0,0.0,0.004077,0.780970,0.194396
328496,0.008250,0.821361,0.029434,0.035344,0.052505,0.022543,0.028935,0.000000,0.0,0.000000,...,0.001826,0.000000,0.0,0.000000,0.0,0.0,0.0,0.003975,0.845209,0.140651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258138,0.014912,0.651536,0.100772,0.091308,0.048283,0.082615,0.009694,0.000000,0.0,0.000000,...,0.001667,0.000000,0.0,0.000000,0.0,0.0,0.0,0.011940,0.779328,0.196800
113312,0.018504,0.701383,0.075905,0.067450,0.040860,0.085433,0.001339,0.007568,0.0,0.001544,...,0.001499,0.000000,0.0,0.000000,0.0,0.0,0.0,0.013264,0.814840,0.154402
185055,0.001051,0.714533,0.058424,0.059715,0.071095,0.032912,0.050922,0.008169,0.0,0.003168,...,0.001680,0.000000,0.0,0.000000,0.0,0.0,0.0,0.009563,0.823618,0.159555
108038,0.000000,0.802671,0.117755,0.054706,0.009113,0.012310,0.000000,0.002293,0.0,0.000000,...,0.001447,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.920234,0.073887
